In [2]:
from pymongo import MongoClient

In [11]:
c=MongoClient()
oa=c["openalexco"]["works"]
kh=c["kahi_dev"]["works"]
khp=c["kahi_dev"]["person"]

In [4]:
pipeline = [
    {"$unwind": "$authors"},

    {
        "$addFields": {
            "authors.affiliations": {
                "$filter": {
                    "input": "$authors.affiliations",
                    "as": "aff",
                    "cond": {
                        "$in": [
                            {"source": "ror", "type": "Education"},
                            "$$aff.types"
                        ]
                    }
                }
            }
        }
    },

    {
        "$match": {
            "$expr": {"$gt": [{"$size": "$authors.affiliations"}, 1]}
        }
    },

    {"$replaceRoot": {"newRoot": "$$ROOT"}},
    {
        "$project": {
            "_id": 1,
            "authors": 1,
            "titles": 1,
            "external_ids": {
                "$filter": {
                    "input": "$external_ids",
                    "as": "ext",
                    "cond": {"$eq": ["$$ext.source", "openalex"]}
                }
            }
        }
    }

]

ws=kh.aggregate(pipeline)

In [6]:
ws=list(ws)

In [7]:
len(ws)

377010

In [8]:
ws[0]

{'_id': ObjectId('68ade991a6d1f814a32696ac'),
 'titles': [{'title': 'Política pública intersectorial a partir de la planificación cultural. Economía, ciencia y tecnología en la planeación cultural',
   'lang': 'es',
   'source': 'openalex'},
  {'title': "Polı́tica pública intersectorial a partir de la planificación cultural. Econom\\á, ciencia y tecnolog\\' ́en la planeación cultural.",
   'lang': 'es',
   'source': 'scholar'},
  {'title': 'Política pública intersectorial a partir de la planificación cultural. Economía, ciencia y tecnología en la planeación cultural',
   'lang': 'es',
   'source': 'minciencias'},
  {'title': 'Política pública intersectorial a partir de la planificación cultural. Economía, ciencia y tecnología en la planeación cultural.',
   'lang': 'es',
   'source': 'dspace'},
  {'title': 'Intersectoral public policy from cultural planning. Economy, science and technology in the cultural planning.',
   'lang': 'en',
   'source': 'dspace'}],
 'authors': {'id': '0000782

In [39]:
def get_oa_author_id(kid):
    pipe=[{
        "$match":{"_id":kid}
    },
        {
        "$project": {
            "external_ids": {
                "$filter": {
                    "input": "$external_ids",
                    "as": "ext",
                    "cond": {"$eq": ["$$ext.source", "openalex"]}
                }
            }
        }
    }
    ]
    _id = list(khp.aggregate(pipe))
    return _id[0]["external_ids"][0]["id"] if _id and _id[0]["external_ids"] else None


In [36]:
ws[0]["authors"]["id"]

'0000782025'

'https://openalex.org/A5036948704'

In [41]:
for w in ws[0:10]:
    _id = get_oa_author_id(w["authors"]["id"])
    fw=oa.find_one({"id":w["external_ids"][0]["id"]}),{"authorship":1}
    print(_id,fw)
    

https://openalex.org/A5036948704 ({'_id': ObjectId('680c485c61c8aec06aa974a3'), 'id': 'https://openalex.org/W101035605', 'doi': 'https://doi.org/10.17227/2011804x.442', 'doi_registration_agency': None, 'display_name': 'Política pública intersectorial a partir de la planificación cultural. Economía, ciencia y tecnología en la planeación cultural', 'title': 'Política pública intersectorial a partir de la planificación cultural. Economía, ciencia y tecnología en la planeación cultural', 'publication_year': 2010, 'publication_date': '2010-07-12', 'language': 'en', 'language_id': 'https://openalex.org/languages/en', 'ids': {'openalex': 'https://openalex.org/W101035605', 'doi': 'https://doi.org/10.17227/2011804x.442', 'mag': 101035605}, 'primary_location': {'source': {'id': 'https://openalex.org/S2736717198', 'issn_l': '2011-804X', 'issn': ['2011-804X', '2462-8441'], 'display_name': 'Pensamiento palabra y obra', 'publisher': 'National Pedagogic University', 'host_organization': 'https://open